In [1]:
# pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# featuretools for automated feature engineering
import featuretools as ft

In [2]:
# # Read in the datasets and replace the anomalous values
# app_train = pd.read_csv('input/application_train.csv').replace({365243: np.nan})
# app_test = pd.read_csv('input/application_test.csv').replace({365243: np.nan})
# bureau = pd.read_csv('input/bureau.csv').replace({365243: np.nan})
# bureau_balance = pd.read_csv('input/bureau_balance.csv').replace({365243: np.nan})
# cash = pd.read_csv('input/POS_CASH_balance.csv').replace({365243: np.nan})
# credit = pd.read_csv('input/credit_card_balance.csv').replace({365243: np.nan})
# previous = pd.read_csv('input/previous_application.csv').replace({365243: np.nan})
# installments = pd.read_csv('input/installments_payments.csv').replace({365243: np.nan})

In [3]:
# app_test['TARGET'] = np.nan

# # Join together training and testing
# app = pd.concat([app_train, app_test], ignore_index = True)
# app = app.sort_index()

In [4]:
# for index in ['SK_ID_CURR', 'SK_ID_PREV', 'SK_ID_BUREAU']:
#     for dataset in [app, bureau, bureau_balance, cash, credit, previous, installments]:
#         if index in list(dataset.columns):
#             dataset[index] = dataset[index].fillna(0).astype(np.int64)

In [5]:
# app.to_csv('./input/application_full.csv', index=False)
# app_train.to_csv('input/application_train.csv', index=False)
# app_test.to_csv('input/application_test.csv', index=False)
# bureau.to_csv('input/bureau.csv', index=False)
# bureau_balance.to_csv('input/bureau_balance.csv', index=False)
# cash.to_csv('input/POS_CASH_balance.csv', index=False)
# credit.to_csv('input/credit_card_balance.csv', index=False)
# previous.to_csv('input/previous_application.csv', index=False)
# installments.to_csv('input/installments_payments.csv', index=False)

In [6]:
tables = ['input/application_full.csv', 'input/bureau.csv','input/bureau_balance.csv','input/POS_CASH_balance.csv',
          'input/credit_card_balance.csv','input/previous_application.csv','input/installments_payments.csv',]

In [7]:
from gen_features import MultiDataFramesFeatureGenerator
generator = MultiDataFramesFeatureGenerator()

In [8]:
generator.load_dataframes(tables)

Loading tables...


100%|██████████| 7/7 [00:22<00:00,  3.14s/it]


Infering Dtypes...


100%|██████████| 7/7 [01:02<00:00,  8.93s/it]


In [9]:
generator.dataframes

[        SK_ID_CURR  TARGET  NAME_CONTRACT_TYPE CODE_GENDER  FLAG_OWN_CAR  \
 0           100002       1                True           M         False   
 1           100003       0                True           F         False   
 2           100004       0                True           M          True   
 3           100006       0                True           F         False   
 4           100007       0                True           M         False   
 ...            ...     ...                 ...         ...           ...   
 356250      456221    <NA>                True           F         False   
 356251      456222    <NA>                True           F         False   
 356252      456223    <NA>                True           F          True   
 356253      456224    <NA>                True           M         False   
 356254      456250    <NA>                True           F          True   
 
         FLAG_OWN_REALTY  CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  \
 0

In [10]:
generator.rename_dataframe(0, 'app')
generator.rename_dataframe(3, 'cash')
generator.rename_dataframe(4, 'credit')
generator.rename_dataframe(5, 'previous')
generator.rename_dataframe(6, 'installments')

In [11]:
generator.set_index(0, 'SK_ID_CURR')
generator.set_index(1, 'SK_ID_BUREAU')
generator.set_index(5, 'SK_ID_PREV')

In [12]:
relationships = [
    ('app', 'SK_ID_CURR', 'bureau', 'SK_ID_CURR',),
	('bureau', 'SK_ID_BUREAU', 'bureau_balance', 'SK_ID_BUREAU',),
	('app', 'SK_ID_CURR', 'previous', 'SK_ID_CURR',),
	('previous', 'SK_ID_PREV', 'cash', 'SK_ID_PREV',),
	('previous', 'SK_ID_PREV', 'installments', 'SK_ID_PREV',),
	('previous', 'SK_ID_PREV', 'credit', 'SK_ID_PREV',),
]
[generator.add_relationship(r) for r in relationships]

[None, None, None, None, None, None]

In [13]:
generator.relationships

[('app', 'SK_ID_CURR', 'bureau', 'SK_ID_CURR'),
 ('bureau', 'SK_ID_BUREAU', 'bureau_balance', 'SK_ID_BUREAU'),
 ('app', 'SK_ID_CURR', 'previous', 'SK_ID_CURR'),
 ('previous', 'SK_ID_PREV', 'cash', 'SK_ID_PREV'),
 ('previous', 'SK_ID_PREV', 'installments', 'SK_ID_PREV'),
 ('previous', 'SK_ID_PREV', 'credit', 'SK_ID_PREV')]

In [14]:
generator.compile()

creating index for column bureau_balance
creating index for column cash
creating index for column credit
creating index for column installments


In [15]:
generator._es.dataframes[0].ww

,Physical Type,Logical Type,Semantic Tag(s)
Column,,,
SK_ID_CURR,int64,Integer,['index']
TARGET,Int64,IntegerNullable,['numeric']
NAME_CONTRACT_TYPE,bool,Boolean,[]
CODE_GENDER,category,Categorical,['category']
FLAG_OWN_CAR,bool,Boolean,[]
FLAG_OWN_REALTY,bool,Boolean,[]
CNT_CHILDREN,int64,Integer,['numeric']
AMT_INCOME_TOTAL,float64,Double,['numeric']
AMT_CREDIT,float64,Double,['numeric']


In [16]:
import pandas as pd
import woodwork
from gen_features import ww_init


df = pd.DataFrame({"a": [1,2,3,4], 'b': [True, False, False,None],
                   'c': [15, 15, 12, 15,],
                   't': ['2023-01-01', '2019-01-23', '2021-12-12', '2022-12-11']})
# df.t = pd.to_datetime(df.t, format='%Y-%m-%d').dt.strftime("%d-%m-%Y")

# from autogluon.features.generators import AutoMLPipelineFeatureGenerator, DropUniqueFeatureGenerator
# ag_dtype_infer =AutoMLPipelineFeatureGenerator(post_generators=[DropUniqueFeatureGenerator(1.0)], post_drop_duplicates=False)
# ag_dtype_infer.fit_transform(df)
# df.ww.init()
ww_init(df, 'abcdf')
df

,a,b,c,t
0,1,True,True,2023-01-01
1,2,False,True,2019-01-23
2,3,False,True,2021-12-12
3,4,NaN,True,2022-12-11


In [17]:
generator.plot('./relationships.svg')

## Run Full Deep Feature Synthesis

If we are content with the features that will be built, we can run deep feature synthesis and create the feature matrix. The following call runs the full deep feature synthesis. This might take a long time depending on your machine. Featuretools does allow for parallel processing, but each core must be able to handle the entire entityset. 

__An actual run of this code was completed using Dask which can be seen in the [Featuretools on Dask notebook](https://github.com/Featuretools/Automated-Manual-Comparison/blob/master/Loan%20Repayment/notebooks/Featuretools%20on%20Dask.ipynb).__ The Dask code takes under 2 hours to run and is a great example of how we can use parallel processing to use our resouces in the most efficient manner.

In [18]:
import sys
print('Total size of entityset: {:.5f} gb.'.format(sys.getsizeof(generator._es) / 1e9))

Total size of entityset: 4.28069 gb.


In [19]:
import psutil

print('Total number of cpus detected: {}.'.format(psutil.cpu_count()))
print('Total size of system memory: {:.5f} gb.'.format(psutil.virtual_memory().total / 1e9))

Total number of cpus detected: 12.
Total size of system memory: 24.92599 gb.


In [20]:
agg_primitives =  ["mean","percent_true", ]
trans_primitives = ['percentile', 'and']

In [21]:
# import time
# time.sleep(1e5)

In [22]:
label = 'TARGET'

In [23]:
result_df = generator.generate_features('app', max_depth=2, verbose=True, chunk_size=50,
                                        # agg_primitives=agg_primitives, trans_primitives=trans_primitives,
                                        ignore_columns={'app': [label]})

/home/minhduck/Documents/projects/vnstock/venv/lib/python3.10/site-packages/featuretools/synthesis/dfs.py:321: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  trans_primitives: ['day', 'month', 'weekday']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible columns for the primitive were found in the data. If the DFS call contained multiple instances of a primitive in the list above, none of them were used.
  warnings.warn(warning_msg, UnusedPrimitiveWarning)


Built 1727 features
Elapsed: 2:25:33 | Progress: 100%|██████████


In [24]:
result_df.shape

(356255, 1727)

In [25]:
result_df.columns

Index(['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
       'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY',
       'AMT_GOODS_PRICE', 'NAME_TYPE_SUITE',
       ...
       'PERCENTILE(SUM(previous.DAYS_FIRST_DUE))',
       'PERCENTILE(SUM(previous.DAYS_LAST_DUE))',
       'PERCENTILE(SUM(previous.DAYS_LAST_DUE_1ST_VERSION))',
       'PERCENTILE(SUM(previous.DAYS_TERMINATION))',
       'PERCENTILE(SUM(previous.HOUR_APPR_PROCESS_START))',
       'PERCENTILE(SUM(previous.NFLAG_INSURED_ON_APPROVAL))',
       'PERCENTILE(SUM(previous.RATE_DOWN_PAYMENT))',
       'PERCENTILE(SUM(previous.RATE_INTEREST_PRIMARY))',
       'PERCENTILE(SUM(previous.RATE_INTEREST_PRIVILEGED))',
       'PERCENTILE(SUM(previous.SELLERPLACE_AREA))'],
      dtype='object', length=1727)

In [26]:
result_df[label] = generator.dataframes[0][label]

In [27]:
df_train = result_df[~result_df[label].isna()]
df_test = result_df[result_df[label].isna()]

In [28]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df_train.drop(columns=[label]), df_train[label], train_size=0.9, random_state=42)

df_train = pd.concat([X_train, y_train], axis=1)
df_val = pd.concat([X_val, y_val], axis=1)

In [29]:
df_train.to_csv('train.csv')
df_val.to_csv('val.csv')
df_test.to_csv('test.csv')


# Feature Selection

In [1]:
from autogluon.tabular import TabularPredictor
from datetime import datetime
import pandas as pd
import featurewiz



df_train= pd.read_csv('train.csv')
df_val= pd.read_csv('val.csv')
df_test= pd.read_csv('test.csv')
label = 'TARGET'
id_cols = ['SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV',]

2024-02-01 21:15:00.953542: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-01 21:15:01.112178: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-01 21:15:01.112227: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-01 21:15:01.140651: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-01 21:15:01.191307: I tensorflow/core/platform/cpu_feature_guar

Imported lazytransform v1.14. 

Imported featurewiz 0.5.4. Use the following syntax:
    >>> wiz = FeatureWiz(feature_engg = '', nrows=None, transform_target=True, scalers="std",
        		category_encoders="auto", add_missing=False, verbose=0. imbalanced=False,
        		ae_options={})
    >>> X_train_selected, y_train = wiz.fit_transform(X_train, y_train)
    >>> X_test_selected = wiz.transform(X_test)
    >>> selected_features = wiz.features
    


In [2]:
X_train, y_train = df_train.drop(columns=[label, *id_cols], errors='ignore'), df_train[label]
X_val, y_val = df_val.drop(columns=[label, *id_cols], errors='ignore'), df_val[label]
X_test, y_test = df_test.drop(columns=[label, *id_cols], errors='ignore'), df_test[label]

df_wiz = pd.concat([df_train, df_val], ignore_index=True)
X_wiz, y_wiz = df_wiz.drop(columns=[label, *id_cols], errors='ignore'), df_wiz[label]

In [3]:
wiz = featurewiz.FeatureWiz(feature_engg = '', nrows=None, transform_target=True, scalers="std",
        		category_encoders="auto", add_missing=False, verbose=0, imbalanced=False, 
                ae_options={})

X_wiz_transform, y_wiz = wiz.fit_transform(X_wiz, y_wiz)


featurewiz is given 0.9 as correlation limit...
    Skipping feature engineering since no feature_engg input...
    final list of category encoders given: ['onehot', 'label']
    final list of scalers given: [std]
Loaded input data. Shape = (307511, 1727)
#### Starting featurewiz transform for train data ####
    Single_Label Binary_Classification problem 
Shape of dataset: (307511, 1727). Now we classify variables into different types...
Time taken to define data pipeline = 601 second(s)
No model input given...
Lazy Transformer Pipeline created...
    transformed target from object type to numeric
    Time taken to fit dataset = 660 second(s)
    Time taken to transform dataset = 695 second(s)
    Shape of transformed dataset: (307511, 2269)
    Single_Label Binary_Classification problem 
Starting SULOV with 2172 features...
    there are no null values in dataset...
    there are no null values in target column...
Completed SULOV. 1057 features selected
Performing recursive XGBoost f

In [4]:
X_train.shape, X_val.shape, X_wiz.shape

((276759, 1727), (30752, 1727), (307511, 1727))

In [5]:
X_train_transform = wiz.transform(X_train)
X_val_transform = wiz.transform(X_val)
X_test_transform = wiz.transform(X_test)

#### Starting featurewiz transform for test data ####
Loaded input data. Shape = (276759, 1727)
#### Starting lazytransform for test data ####
    Time taken to transform dataset = 25 second(s)
    Shape of transformed dataset: (276759, 2269)
Returning dataframe with 354 features 
#### Starting featurewiz transform for test data ####
Loaded input data. Shape = (30752, 1727)
#### Starting lazytransform for test data ####
    Time taken to transform dataset = 2 second(s)
    Shape of transformed dataset: (30752, 2269)
Returning dataframe with 354 features 
#### Starting featurewiz transform for test data ####
Loaded input data. Shape = (48744, 1727)
#### Starting lazytransform for test data ####
    Time taken to transform dataset = 3 second(s)
    Shape of transformed dataset: (48744, 2269)
Returning dataframe with 354 features 


In [6]:
for col in id_cols:
	if col not in df_train.columns: continue
	X_train_transform[col] = df_train[col]
	X_val_transform[col] = df_val[col]
	X_test_transform[col] = df_test[col]


train_df = pd.concat([X_train_transform, y_train], axis=1)
val_df = pd.concat([X_val_transform, y_val], axis=1)
test_df = pd.concat([X_test_transform, y_test], axis=1)

In [12]:
train_df.to_csv('train_pruned.csv')
val_df.to_csv('val_pruned.csv')
test_df.to_csv('test_pruned.csv')

# Training


In [6]:
from autogluon.tabular import TabularPredictor
from datetime import datetime
import pandas as pd


train_df = pd.read_csv('train_pruned.csv')
val_df = pd.read_csv('val_pruned.csv')
test_df = pd.read_csv('test_pruned.csv')
label = "TARGET"

In [2]:
from autogluon.tabular.configs.hyperparameter_configs import get_hyperparameter_config
get_hyperparameter_config('multimodal')

{'NN_TORCH': {},
 'GBM': [{},
  {'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}},
  'GBMLarge'],
 'CAT': {},
 'XGB': {},
 'AG_AUTOMM': {},
 'VW': {}}

In [3]:

kwargs = {
    'time_limit': 7200,
    'presets': 'medium_quality',
    'hyperparameters': {
        'GBM': [{'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}},
                 {},
                #  {'boosting': 'dart', 'ag_args': {'name_suffix': 'dart'}},
                 'GBMLarge'
                ],
         'CAT': {},
         'XGB': {},
        'AG_AUTOMM': {}, 'VW': {}
    },
	"ag_args_fit":{"ag.max_memory_usage_ratio": 6.0},
	"ag_args_ensemble":{"fold_fitting_strategy":"sequential_local"},
}
predictor = TabularPredictor(label, eval_metric='roc_auc')
predictor.fit(
        train_df, tuning_data=val_df,
        # pd.concat([train_df, val_df], ignore_index=True), 
              **kwargs)

No path specified. Models will be saved in: "AutogluonModels/ag-20240226_134632"
Presets specified: ['medium_quality']
Beginning AutoGluon training ... Time limit = 7200s
AutoGluon will save models to "AutogluonModels/ag-20240226_134632"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #21~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Fri Feb  9 13:32:52 UTC 2
CPU Count:          12
Memory Avail:       14.42 GB / 23.21 GB (62.1%)
Disk Space Avail:   368.76 GB / 465.76 GB (79.2%)
Train Data Rows:    276759
Train Data Columns: 356
Tuning Data Rows:    30752
Tuning Data Columns: 356
Label Column:       TARGET
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may spec

[1000]	valid_set's binary_logloss: 0.241386


	0.7806	 = Validation score   (roc_auc)
	291.37s	 = Training   runtime
	0.87s	 = Validation runtime
Fitting model: LightGBM ... Training model for up to 6901.08s of the 6901.08s of remaining time.
	0.7811	 = Validation score   (roc_auc)
	228.35s	 = Training   runtime
	0.2s	 = Validation runtime
Fitting model: CatBoost ... Training model for up to 6672.5s of the 6672.5s of remaining time.
	0.7846	 = Validation score   (roc_auc)
	200.86s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: XGBoosttree ... Training model for up to 6471.58s of the 6471.58s of remaining time.
	0.779	 = Validation score   (roc_auc)
	86.76s	 = Training   runtime
	0.27s	 = Validation runtime
Fitting model: XGBoosttree_1 ... Training model for up to 6384.51s of the 6384.51s of remaining time.
	0.7755	 = Validation score   (roc_auc)
	74.6s	 = Training   runtime
	0.34s	 = Validation runtime
Fitting model: XGBoosttree_2 ... Training model for up to 6309.55s of the 6309.55s of remaining time.
	0.7731	 

In [7]:
from autogluon.tabular import TabularPredictor
predictor = TabularPredictor.load("AutogluonModels/ag-20240226_134632")
predictor.leaderboard(val_df, extra_metrics=['accuracy', 'roc_auc', 'precision', 'recall', 'f1']).to_csv('leaderboard.csv')

<class 'inspect._empty'>
<class 'inspect._empty'>
<class 'inspect._empty'>
<class 'inspect._empty'>


Load pretrained checkpoint: /media/minhduck/SSD1/Documents/projects/AutoML-Platform/src/mlservice/app/test/relational_test/AutogluonModels/ag-20240226_134632/models/MultiModalPredictor/automm_model/model.ckpt
You are using a CUDA device ('NVIDIA GeForce RTX 3050 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


Predicting DataLoader 0: 100%|██████████| 961/961 [00:19<00:00, 49.08it/s]


In [4]:
predictor.leaderboard(val_df, extra_metrics=['accuracy', 'roc_auc'])

Load pretrained checkpoint: /media/minhduck/SSD1/Documents/projects/AutoML-Platform/src/mlservice/app/test/relational_test/AutogluonModels/ag-20240226_134632/models/MultiModalPredictor/automm_model/model.ckpt


Predicting DataLoader 0: 100%|██████████| 961/961 [00:18<00:00, 52.36it/s]


,model,score_test,accuracy,roc_auc,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.787219,0.919355,0.787219,0.787219,roc_auc,20.113111,20.052561,6845.866517,0.004305,0.003055,3.039491,2,True,11
1,CatBoost,0.784627,0.919745,0.784627,0.784627,roc_auc,0.057204,0.037113,200.862975,0.057204,0.037113,200.862975,1,True,3
2,LightGBMLarge,0.781362,0.919485,0.781362,0.781362,roc_auc,0.174113,0.153152,65.188701,0.174113,0.153152,65.188701,1,True,9
3,LightGBM,0.781095,0.919030,0.781095,0.781095,roc_auc,0.167712,0.196187,228.354290,0.167712,0.196187,228.354290,1,True,2
4,LightGBMXT,0.780646,0.918965,0.780646,0.780646,roc_auc,0.266048,0.873061,291.365826,0.266048,0.873061,291.365826,1,True,1
5,XGBoosttree,0.779046,0.918997,0.779046,0.779046,roc_auc,0.493690,0.267194,86.758784,0.493690,0.267194,86.758784,1,True,4
6,XGBoosttree_3,0.775896,0.919290,0.775896,0.775896,roc_auc,0.323460,0.312481,96.913277,0.323460,0.312481,96.913277,1,True,7
7,XGBoosttree_1,0.775534,0.918444,0.775534,0.775534,roc_auc,0.319357,0.338408,74.599018,0.319357,0.338408,74.599018,1,True,5
8,XGBoosttree_4,0.775345,0.918022,0.775345,0.775345,roc_auc,0.313715,0.272978,66.894486,0.313715,0.272978,66.894486,1,True,8
9,XGBoosttree_2,0.773050,0.918282,0.773050,0.773050,roc_auc,0.324655,0.348277,69.756819,0.324655,0.348277,69.756819,1,True,6


In [5]:
prediction = predictor.predict_proba(test_df, model=None)

Load pretrained checkpoint: /media/minhduck/SSD1/Documents/projects/AutoML-Platform/src/mlservice/app/test/relational_test/AutogluonModels/ag-20240226_134632/models/MultiModalPredictor/automm_model/model.ckpt


Predicting DataLoader 0: 100%|██████████| 1524/1524 [00:29<00:00, 52.07it/s]


In [ ]:
test_df['TARGET'] = prediction[1]

In [ ]:
test_df[['SK_ID_CURR', 'TARGET']].to_csv('sub.csv', index=False)